In [7]:
import Utils.FS.file as file

In [11]:
eng_link_posts = file.read('eng_link_posts_2.pkl')

In [12]:
eng_link_posts.shape

(7695711, 23)

In [14]:
eng_link_posts.sample(100)

,name,caption,message,description,status_type,type,link,full_picture,picture,source,...,page_id,share,comment,like,reaction,love,haha,wow,sad,angry
7342280,Reading Comprehension – How to Stop Procrastin...,abaenglish.com,Check out today's post to learn 5 tips to figh...,"Do you put the ""pro"" in ""procrastinate""? In ca...",shared_story,link,http://www.abaenglish.com/blog/reading-compreh...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,22689541360,22,1,265,265,NaN,NaN,NaN,NaN,NaN
210773,Hitting the Apex: Marlon Stockinger,megastyle.ph,There's more to Marlon Stockinger than meets t...,"“If you have everything under control, you’re ...",shared_story,link,http://megastyle.ph/hitting-the-apex-marlon-st...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,63464009352,1,0,195,197,NaN,NaN,NaN,NaN,NaN
4118107,Marlen Garcia: Defeating Obama on immigration ...,chicago.suntimes.com,http://chicago.suntimes.com/opinion/7/71/10956...,"Follow @marlengarcia777 For the last year, con...",shared_story,link,http://chicago.suntimes.com/opinion/7/71/10956...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,91052833204,69,28,1496,1496,NaN,NaN,NaN,NaN,NaN
5035875,Quiosque: o adeus do FC Porto à Champions e o ...,maisfutebol.iol.pt,Bom dia. Já viu o que dizem os jornais que est...,NaN,shared_story,link,https://goo.gl/1k9dYs,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,343414416186,0,1,22,22,NaN,NaN,NaN,NaN,NaN
3145512,Brian Skerry on Twitter,twitter.com/Brian_Skerry/s…,Always beautiful photos from Brian.... https:/...,“10 Intimate Photos for Dolphin Day https://t....,shared_story,link,https://t.co/XJZlSO739Y,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,194586413405,0,0,80,81,NaN,NaN,NaN,NaN,NaN
1123119,Parquet EP,soundcloud.com,SoundCloud has reinstalled my two tracks they ...,Listen to Parquet EP by Sébastien Léger #np on...,shared_story,link,https://soundcloud.com/sebastienleger/sets/par...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,22828809088,7,9,164,176,NaN,NaN,NaN,NaN,NaN
4804678,Riot Festivus Holiday Display - Riot Fest,riotfest.org,You can win 6 VIP tickets to Riot Fest Chicago...,Do you want to get in the holiday spirit and h...,shared_story,link,http://riotfest.org/2016/12/riot-festivus-holi...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,37054718736,0,2,11,13,NaN,NaN,NaN,NaN,NaN
1289992,Disney thanks you: 100 million page Likes,gdata.youtube.com,We can think of no greater birthday present fo...,Thank you to all our fans who shared in the ma...,shared_story,link,http://www.youtube.com/watch?v=bVQ4bskC6D8,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,135040752614,7,79,558,558,NaN,NaN,NaN,NaN,NaN
1252324,Change Your Clock Change Your Battery | Energizer,energizer.com,It’s #FireSafetyMonth! We’re teaming up with l...,Change your batteries on Nov. 2 and help Energ...,shared_story,link,http://energizer.com/cyccyb,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,64260663696,12,16,24,24,NaN,NaN,NaN,NaN,NaN
1875131,$279 -- Summer Return Flts to Osaka ex SG w/Ta...,travelzoo.com,Book your flights early for your Osaka summer ...,Don't miss the city's historic Tenjin Matsuri ...,shared_story,link,http://www.travelzoo.com/ae/-279-Summer-Return...,https://external.xx.fbcdn.net/safe_image.php?d...,https://external.xx.fbcdn.net/safe_image.php?d...,NaN,...,349794865146821,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [3]:
NROWS = None

pages = pd.read_csv('./pages.csv', nrows=NROWS)
pages = pages[['id', 'fan_count']]

In [4]:
result = pd.merge(en_link_posts, pages, left_on='page_id', right_on='id')

In [5]:
result['share_ratio'] = result['share'] / result['fan_count']
result['normalized_share_ratio'] = result['share_ratio'] / result['share_ratio'].max()
print(result['normalized_share_ratio'].describe())
data = result[['name', 'normalized_share_ratio']]

count    990127.000000
mean          0.000442
std           0.002739
min           0.000000
25%           0.000005
50%           0.000037
75%           0.000190
max           1.000000
Name: normalized_share_ratio, dtype: float64


In [6]:
import pickle
with open('clean_list_post_name.pkl', 'rb') as f:
    clean_train_name = pickle.load(f)

In [7]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_name)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [8]:
vocab = vectorizer.get_feature_names()

In [9]:
import numpy as np
train_X = train_data_features[:300000, :]
train_y = data["normalized_share_ratio"].values[:300000]

import xgboost as xgb


xgb_params = {"objective": "reg:linear", "eta": 0.01, "max_depth": 10, "seed": 42, "silent": 1, "booster":"gblinear"}
num_rounds = 100

dtrain = xgb.DMatrix(train_X, label=train_y, feature_names=vocab)
gbm = xgb.train(xgb_params, dtrain, num_rounds)

dtest = xgb.DMatrix(train_X, feature_names=vocab)

print(gbm.predict(dtest))

print(gbm.get_score())
print(gbm.get_fscore())

In [13]:
import xgboost as xgb
gbm = xgb.XGBRegressor().fit(train_X, train_y)

In [12]:
importances = list(zip(vocab, gbm.feature_importances_))
importances.sort(key = lambda t: t[1], reverse=True)
print(importances[:100])

AttributeError: 'Booster' object has no attribute 'feature_importances_'